# Export, Import, Clean, Merge Tutorial

The purpose of this exercise is to illustrate several wrangling concepts covered in class and in DataCamp. We begin by “messing up” the diamonds data then put it back together. During this exercise, you will see imports, exports, reshapes, joins, mutations, and more!

Note: This notebook is intended to be interactive and contains incomplete code. See [solution file](diamonds_wrangle_exercise_solution.ipynb) in this repository for the final solution and output.

## Housekeeping

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Deconstruct the diamonds dataset
Open and store the diamonds dataset after applying a filter on carat weight (less than 1.5 carats) and keeping only carat, clarity, and price. Also, let's create a unique identifier (`uniq_id`) for each diamond. Information about the original dataset can be found [here](https://ggplot2.tidyverse.org/reference/diamonds.html). Store in a new dataset called `diamonds_subset`. This will be our starting point for the exercise.

In [ ]:
diamonds = sns.load_dataset("diamonds")
# diamonds = pd.read_pickle('diamonds.pkl')

diamonds_subset = (
    diamonds[diamonds['carat'] < 1.5][['carat', 'clarity', 'price']]
    .assign(uniq_id=lambda d: range(1, len(d) + 1)) # add a unique identifier
    .reset_index(drop=True)
)

Take a look at a sample of the data:

In [ ]:
diamonds_subset.sample(10, random_state=42)

Select the `uniq_id` and `price` variables, rename the `uniq_id` as `id`, and add extra characters to `price`. Create a new dataset called `diamonds_prices`:

In [ ]:
diamonds_prices = (
    diamonds_subset[['uniq_id', 'price']]
    .rename(columns={'uniq_id': 'id'})
)

diamonds_prices['price'] = 'amount ' + diamonds_prices['price'].astype(str) + ' USD'

Add a few duplicated observations to `diamonds_prices` using a random draw from `diamonds_prices`:

In [ ]:
sampled = diamonds_prices.sample(55, random_state=42) # Take a random sample of 55 rows

diamonds_prices = (
    pd.concat([diamonds_prices, sampled]) # Append the sampled rows to the original diamonds_prices dataframe
    .sort_values('id')
)

del sampled

Next, let's store the other diamond characteristics (i.e., carat and clarity) in another 'messy' dataset. Select the diamond characteristics excluding `price` and reshape to long format. Store as a new dataset - `diamonds_char_long`:

In [ ]:
diamonds_char = (
    diamonds_subset
    .drop(columns='price')
    .melt(id_vars='uniq_id', var_name='variable', value_name='measure')
    .sort_values(['uniq_id', 'variable'])
)

Save each of the new datasets in different formats (csv and txt) and delete all variables/data from the workspace:

In [ ]:
diamonds_prices.to_csv('diamonds_prices.csv', index=False)
diamonds_char.to_csv('diamonds_char.txt', sep='\t', index=False)

del diamonds, diamonds_subset, diamonds_prices, diamonds_char

## Reconstruct the diamonds dataset
Take a look at your working directory (i.e., the current folder we are working in). You should have two files saved from above. These files include all of the information from the original dataset (`diamonds_subset`), but the data is now in raw and messy form ☹️. Let's imagine that we are starting from scratch with only these files and need to reconstruct the diamonds dataset. Before we begin, find the files on your local computer and open them. To navigate to the file location, right-click on the file/folder and select 'open in' then select explorer (or finder on a Mac).

Let’s begin by importing each dataset as `diamonds_prices_import` and `diamonds_char_import`:

In [ ]:
diamonds_prices_import =
diamonds_char_import =

Take a look at a sample of the `diamonds_prices` data.

As you can see, the price variable needs to be cleaned. We need to remove the text and keep only the price. Let’s tackle it!

In [ ]:
# Copy diamonds_prices_import in diamonds_prices for cleaning
diamonds_prices = diamonds_prices_import.copy()

# Remove "amount " from the price column

# Split the price column into two columns: the price and currency

# Convert the numeric price (currently a string) into numeric type


diamonds_prices.sample(10, random_state=42)

Recall that we added some duplicates to the prices dataset… let’s check for duplicates and store in a variable called `duplicates`.

In [ ]:
duplicates =

duplicates.head()

This reports the duplicated observations in the dataset. Take a look at the `duplicates` dataframe. There are 110 rows and 55 duplicated observations. A quick inspection suggests that the duplicates are exact copies of each other. We can remove the duplicates by keeping only the first occurrence of each duplicated `id`.

In [ ]:
diamonds_prices =

Next, we need to work with the diamonds_char_import dataset. Let's start by inspecting the dataset:

In [ ]:
diamonds_char_import.head(8)

At first glance, this dataset seems to have many duplicate values for `uniq_id`. However, a closer look suggests that each row contains different information about each diamond. Remember what we learned about *tidy data*... Each observation (i.e., diamond) should be in a row, and each feature (i.e., carat and clarity) should be a column. To get there, the dataset needs to be reshaped from long to wide. Let’s do that now.

In [ ]:
# Copy diamonds_char_import in diamonds_char for cleaning
diamonds_char = diamonds_char_import.copy()

# Reshape the diamonds_char dataset from long to wide


diamonds_char.head(4)

That’s better. However, all formatting from the variables is lost. In particular, we need to convert carat to a numeric variable and clarity to an ordered categorical (or factor) variable:

In [ ]:
diamonds_char['carat'] =

diamonds_char['clarity'] =

diamonds_char.head(4)

As a sanity check, let’s inspect the dataset for duplicate observations:



In [ ]:
#Check for duplicates in the reshaped dataset
duplicates =

duplicates.head()

OK, no duplicates this time… we are finally ready to join! Remember that we tracked each diamond with a unique identifier. We will use that to reconnect the two datasets.

In [ ]:
diamonds_recon =

diamonds_recon.head()

Let’s plot our reconstructed dataset:



In [ ]:
sns.relplot(data=diamonds_recon,
            x='carat',
            y='price',
            hue='clarity',
            kind='scatter',
            alpha=0.3,
            palette=sns.color_palette('magma', n_colors=len(diamonds_recon['clarity'].unique()))
            )

plt.show()